In [1]:
import fcalc
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
import time
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('data_sets/iris.data', names=['sepal_length',	'sepal_width',	'petal_length',	'petal_width','species'])
le = LabelEncoder(); le.fit(df["species"].values)
df["species"] = le.transform(df["species"].values)

In [3]:
def model_test_CV(X, y, cat_cols=None, method="standard", alpha=0.,
                  randomize=False, num_iters=10, subsample_size=1e-2,
                  kde_bandwidth=1.0, kde_kernel='gaussian', kde_leaf_size=40,
                  kde_classwise=False, scale_density=True, n_splits=5, seed=42):
    kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)
    Accuracy = []
    F1_score = []
    exec_time = []
    for train_index, test_index in tqdm(kf.split(X), total=kf.get_n_splits(), desc="k-fold"):
        pat_cls = fcalc.classifier.PatternClassifier(X[train_index], y[train_index], 
                                                     categorical=cat_cols, method=method,
                                                     alpha=alpha, randomize=randomize, num_iters=num_iters,
                                                     subsample_size=subsample_size, kde_bandwidth=kde_bandwidth,
                                                     kde_kernel=kde_kernel, kde_leaf_size=kde_leaf_size,
                                                     kde_classwise=kde_classwise, scale_density=scale_density)
        start = time.time()
        pat_cls.predict(X[test_index])
        end = time.time()
        Accuracy.append(round(accuracy_score(y[test_index], pat_cls.predictions),4))
        F1_score.append(round(f1_score(y[test_index], pat_cls.predictions, average='macro'),4))
        exec_time.append(round(end-start, 4))
    
    Accuracy.append(np.mean(Accuracy)); F1_score.append(np.mean(F1_score)); exec_time.append(np.mean(exec_time))
    return pd.DataFrame(zip(Accuracy,F1_score,exec_time),
                        columns=["Accuracy","F1 score", "time (s)"],
                        index=[x+1 for x in range(kf.get_n_splits())]+["mean"])

In [5]:
X_train = df.drop('species', axis=1).values
y_train = df['species'].values
n_iters = np.arange(10,51,10)
s_size = np.linspace(1e-2,5e-2,num=5,endpoint=True)
result = []
for n in n_iters:
    for s in s_size:
        res = model_test_CV(X_train,y_train,randomize=True,num_iters=n,subsample_size=s)
        result.append(res.loc["mean"].values)
result=pd.DataFrame(result,columns=["Accuracy","F1 score", "time (sec.)"], 
                    index=pd.MultiIndex.from_product([n_iters, s_size], names=["Number of iterations","Subsample size"]))
display(result)
result.to_csv("D:/University/masters thesis/csv results/iris_random_res_multiindex.csv", index=True)

k-fold: 100%|██████████| 5/5 [00:00<00:00,  5.14it/s]


Accuracy  F1 score  time (sec.)
Number of iterations Subsample size                                 
10                   0.01             0.82668   0.66740       0.0464
                     0.02             0.82668   0.66740       0.0404
                     0.03             0.90000   0.72664       0.0402
                     0.04             0.90000   0.72664       0.0486
                     0.05             0.88000   0.67626       0.0402
20                   0.01             0.86000   0.67932       0.0850
                     0.02             0.86000   0.67932       0.0792
                     0.03             0.91334   0.86668       0.0792
                     0.04             0.91334   0.86668       0.0784
                     0.05             0.90666   0.73686       0.0792
30                   0.01             0.90666   0.75190       0.1162
                     0.02             0.90666   0.75190       0.1162
                     0.03             0.91334   0.86668       0.1164
                     0.04             0.91334   0.86668       0.1176
                     0.05             0.91334   0.86510       0.1270
40                   0.01             0.91332   0.75214       0.1538
                     0.02             0.91332   0.75214       0.1542
                     0.03             0.91334   0.86750       0.1550
                     0.04             0.91334   0.86750       0.1560
                     0.05             0.91334   0.90934       0.1554
50                   0.01             0.91332   0.75214       0.1908
                     0.02             0.91332   0.75214       0.1892
                     0.03             0.92668   0.88250       0.2016
                     0.04             0.92668   0.88250       0.1912
                     0.05             0.91334   0.86510       0.1916